In [1]:
# From http://aimotion.blogspot.com.es/2013/01/machine-learning-and-data-mining.html

# How to build frequent itemsets?

There is an algorithm to generate frequent itemsets in an efficient way: Apriori

## Apriori principle

```
If an itemset is frequent, all of its subsets are frequent
```

## Reverse of aprori principle
```
If an itemset is infrequent, then all of its supersets are infrequent
```
<img src="files/lat_itemsets.png" style="width:400px; display:inline">

## Finding frequent itemsets
```
Calculate L1 as the frequent itemsets of length 1
k = 2
While L(k-1) is not empty
    Generate the candidate set of length k from L(k-1)
    Scan the candidate set to find frequent itemsets of length k 
    Lk are the frequent itemsets of length k
    k = k + 1
```


# Finding frequent itemsets

In [2]:
#-*- coding:utf-8 - *-

dict_products = {
    1: 'tomate',
    2: 'berenjena',
    3: 'cerveza',
    4: 'gamba', 
    5: 'donut',    
}

def load_dataset():
    "Load the sample dataset."
    #return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
    return [[dict_products[1], dict_products[3], dict_products[4]], 
            [dict_products[2], dict_products[3], dict_products[5]], 
            [dict_products[1], dict_products[2], dict_products[3], dict_products[5]], 
            [dict_products[2], dict_products[5]]]


def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a key of a dictionary.
    return map(frozenset, c1)


def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data


def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList


def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = map(set, dataset)
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1

    return L, support_data

In [3]:
dataset = load_dataset()
dataset

[['tomate', 'cerveza', 'gamba'],
 ['berenjena', 'cerveza', 'donut'],
 ['tomate', 'berenjena', 'cerveza', 'donut'],
 ['berenjena', 'donut']]

In [4]:
C1 = createC1(dataset)
C1

[frozenset({'berenjena'}),
 frozenset({'cerveza'}),
 frozenset({'donut'}),
 frozenset({'gamba'}),
 frozenset({'tomate'})]

In [5]:
D = map(set, dataset)
D

[{'cerveza', 'gamba', 'tomate'},
 {'berenjena', 'cerveza', 'donut'},
 {'berenjena', 'cerveza', 'donut', 'tomate'},
 {'berenjena', 'donut'}]

In [6]:
L1, support_data = scanD(D, C1, 0.5)
print("Frequent itemsets of length 1: {}".format(L1))
support_data

Frequent itemsets of length 1: [frozenset(['berenjena']), frozenset(['donut']), frozenset(['tomate']), frozenset(['cerveza'])]


{frozenset({'gamba'}): 0.25,
 frozenset({'cerveza'}): 0.75,
 frozenset({'tomate'}): 0.5,
 frozenset({'berenjena'}): 0.75,
 frozenset({'donut'}): 0.75}

In [7]:
C2 = aprioriGen(L1,2)
C2

[frozenset({'berenjena', 'donut'}),
 frozenset({'berenjena', 'tomate'}),
 frozenset({'berenjena', 'cerveza'}),
 frozenset({'donut', 'tomate'}),
 frozenset({'cerveza', 'donut'}),
 frozenset({'cerveza', 'tomate'})]

In [8]:
L2, support_data2 = scanD(D, C2, 0.5)
print("Frequent itemsets of length 2: {}".format(L2))
support_data2

Frequent itemsets of length 2: [frozenset(['donut', 'berenjena']), frozenset(['tomate', 'cerveza']), frozenset(['donut', 'cerveza']), frozenset(['cerveza', 'berenjena'])]


{frozenset({'donut', 'tomate'}): 0.25,
 frozenset({'berenjena', 'cerveza'}): 0.5,
 frozenset({'cerveza', 'donut'}): 0.5,
 frozenset({'cerveza', 'tomate'}): 0.5,
 frozenset({'berenjena', 'donut'}): 0.75,
 frozenset({'berenjena', 'tomate'}): 0.25}

In [9]:
L, supportdata = apriori(dataset, minsupport=0.5)
print(L)
supportdata

[[frozenset(['berenjena']), frozenset(['donut']), frozenset(['tomate']), frozenset(['cerveza'])], [frozenset(['donut', 'berenjena']), frozenset(['tomate', 'cerveza']), frozenset(['donut', 'cerveza']), frozenset(['cerveza', 'berenjena'])], [frozenset(['donut', 'cerveza', 'berenjena'])], []]


{frozenset({'tomate'}): 0.5,
 frozenset({'donut'}): 0.75,
 frozenset({'donut', 'tomate'}): 0.25,
 frozenset({'cerveza'}): 0.75,
 frozenset({'berenjena'}): 0.75,
 frozenset({'berenjena', 'cerveza'}): 0.5,
 frozenset({'cerveza', 'donut'}): 0.5,
 frozenset({'cerveza', 'tomate'}): 0.5,
 frozenset({'gamba'}): 0.25,
 frozenset({'berenjena', 'donut'}): 0.75,
 frozenset({'berenjena', 'cerveza', 'donut'}): 0.5,
 frozenset({'berenjena', 'tomate'}): 0.25}

# Mining association rules from frequent item sets

Once we have a way to calculate the frequent item sets, we can generate the association rules that fit the support and confidence requirements.

## Apriori principle in association rules

``` If the rule does not meet the minimum confidence requirement, then rules with the left hand side subset of that rule also will not meet the minimum. ```

<img src="files/lat_rules.png" style="width:400px; display:inline">

## Finding association rules

```
Calculate frequent itemsets
rules = []
For each frequent itemset
    Create the list of rules with 1 item in the right hand side
    Test the confidence of the rules and add the ones whose confidence is higher than the threshold
    Generate the list of rules with 2 items in the right hand side
    k = 2
    While there are candidate rules with k items in the right hand side
        Test the confidence of the rules and add the ones whose confidence is higher than the threshold
        Generate the list of rules with the right hand side of length k + 1
        k = k + 1
```

In [11]:
def generateRules(L, support_data, min_confidence=0.7):
    """Create the association rules
    L: list of frequent item sets
    support_data: support data for those itemsets
    min_confidence: minimum confidence threshold
    """
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            print "freqSet", freqSet, 'H1', H1
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules


def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.7):
    "Evaluate the rule generated"
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            print freqSet - conseq, '--->', conseq, 'conf:', conf
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H

def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    "Generate a set of candidate rules"
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [12]:
dataset = load_dataset()
dataset

[['tomate', 'cerveza', 'gamba'],
 ['berenjena', 'cerveza', 'donut'],
 ['tomate', 'berenjena', 'cerveza', 'donut'],
 ['berenjena', 'donut']]

In [13]:
L, supportdata = apriori(dataset, minsupport=0.5)
print(L)
supportdata

[[frozenset(['berenjena']), frozenset(['donut']), frozenset(['tomate']), frozenset(['cerveza'])], [frozenset(['donut', 'berenjena']), frozenset(['tomate', 'cerveza']), frozenset(['donut', 'cerveza']), frozenset(['cerveza', 'berenjena'])], [frozenset(['donut', 'cerveza', 'berenjena'])], []]


{frozenset({'tomate'}): 0.5,
 frozenset({'donut'}): 0.75,
 frozenset({'donut', 'tomate'}): 0.25,
 frozenset({'cerveza'}): 0.75,
 frozenset({'berenjena'}): 0.75,
 frozenset({'berenjena', 'cerveza'}): 0.5,
 frozenset({'cerveza', 'donut'}): 0.5,
 frozenset({'cerveza', 'tomate'}): 0.5,
 frozenset({'gamba'}): 0.25,
 frozenset({'berenjena', 'donut'}): 0.75,
 frozenset({'berenjena', 'cerveza', 'donut'}): 0.5,
 frozenset({'berenjena', 'tomate'}): 0.25}

In [14]:
rules = generateRules(L, supportdata, min_confidence=0.7)
L
rules

freqSet frozenset(['donut', 'berenjena']) H1 [frozenset(['donut']), frozenset(['berenjena'])]
frozenset(['berenjena']) ---> frozenset(['donut']) conf: 1.0
frozenset(['donut']) ---> frozenset(['berenjena']) conf: 1.0
freqSet frozenset(['tomate', 'cerveza']) H1 [frozenset(['tomate']), frozenset(['cerveza'])]
frozenset(['tomate']) ---> frozenset(['cerveza']) conf: 1.0
freqSet frozenset(['donut', 'cerveza']) H1 [frozenset(['donut']), frozenset(['cerveza'])]
freqSet frozenset(['cerveza', 'berenjena']) H1 [frozenset(['cerveza']), frozenset(['berenjena'])]
freqSet frozenset(['donut', 'cerveza', 'berenjena']) H1 [frozenset(['donut']), frozenset(['cerveza']), frozenset(['berenjena'])]


[(frozenset({'berenjena'}), frozenset({'donut'}), 1.0),
 (frozenset({'donut'}), frozenset({'berenjena'}), 1.0),
 (frozenset({'tomate'}), frozenset({'cerveza'}), 1.0)]